En este dataset se desea pronosticar el precio de vhiculos usados. El dataset
Original contiene las siguientes columnas:

- Car_Name: Nombre del vehiculo.
- Year: Año de fabricación.
- Selling_Price: Precio de venta.
- Present_Price: Precio actual.
- Driven_Kms: Kilometraje recorrido.
- Fuel_type: Tipo de combustible.
- Selling_Type: Tipo de vendedor.
- Transmission: Tipo de transmisión.
- Owner: Número de propietarios.

El dataset ya se encuentra dividido en conjuntos de entrenamiento y prueba
en la carpeta "files/input/".

Los pasos que debe seguir para la construcción de un modelo de
pronostico están descritos a continuación.

In [1]:
# Paso 1.
# Preprocese los datos.
# - Cree la columna 'Age' a partir de la columna 'Year'.
#   Asuma que el año actual es 2021.
# - Elimine las columnas 'Year' y 'Car_Name'.

import pandas as pd

test = pd.read_csv('../files/input/test_data.csv.zip', compression ="zip",)
train = pd.read_csv('../files/input/train_data.csv.zip', compression ="zip",)
train['Age'] = 2021 - train['Year']
test['Age'] = 2021 - test['Year']
test.drop(columns=['Year','Car_Name'], inplace=True)
train.drop(columns=['Year','Car_Name'], inplace=True)




In [2]:

# Paso 2.
# Divida los datasets en x_train, y_train, x_test, y_test.


x_train = train.drop(columns=['Present_Price'])
y_train = train['Present_Price']
x_test = test.drop(columns=['Present_Price'])
y_test = test['Present_Price']
cat_cols = [c for c in x_train.columns if x_train[c].dtype == 'object']
num_cols = [c for c in x_train.columns if x_train[c].dtype != 'object']

In [3]:
#
#
# Paso 3.
# Cree un pipeline para el modelo de clasificación. Este pipeline debe
# contener las siguientes capas:
# - Transforma las variables categoricas usando el método
#   one-hot-encoding.
# - Escala las variables numéricas al intervalo [0, 1].
# - Selecciona las K mejores entradas.
# - Ajusta un modelo de regresion lineal.



from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder


preprocessor = ColumnTransformer([
        ('onehot', OneHotEncoder(handle_unknown='ignore'), cat_cols),
        ('scaler', MinMaxScaler(), num_cols)
    ], remainder='passthrough')

pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('feature_selection', SelectKBest(score_func=f_regression)),
    ('linearR', LinearRegression())
])

In [4]:
#
#
# Paso 4.
# Optimice los hiperparametros del pipeline usando validación cruzada.
# Use 10 splits para la validación cruzada. Use el error medio absoluto
# para medir el desempeño modelo.


from sklearn.model_selection import GridSearchCV


param_grid = {
    'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
}

model = GridSearchCV(
    estimator=pipeline, param_grid=param_grid,
    scoring='neg_mean_absolute_error',
    cv=10, n_jobs=-1,

)

model.fit(x_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('onehot',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         ['Fuel_Type',
                                                                          'Selling_type',
                                                                          'Transmission']),
                                                                        ('scaler',
                                                                         MinMaxScaler(),
                                                                         ['Selling_Price',
                                                                          'Driven_kms',
                                                                          'Owner',
                                                                          'Age'])])),
                                       ('feature_selection',
                                        SelectKBest(score_func=<function f_regression at 0x746efd30dee0>)),
                                       ('linearR', LinearRegression())]),
             n_jobs=-1,
             param_grid={'feature_selection__k': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10,
                                                  11]},
             scoring='neg_mean_absolute_error')

In [5]:
#
#
# Paso 5.
# Guarde el modelo (comprimido con gzip) como "files/models/model.pkl.gz".
# Recuerde que es posible guardar el modelo comprimido usanzo la libreria gzip.


import os
import pickle
import gzip

# Definir la ruta del directorio y archivo
dir_path = '../files/models'
model_path = '../files/models/model.pkl.gz' 

os.makedirs(dir_path, exist_ok=True)

with gzip.open(model_path, 'wb') as  f:

    pickle.dump(model, f)


In [6]:
#
#
# Paso 6.
# Calcule las metricas r2, error cuadratico medio, y error absoluto medio
# para los conjuntos de entrenamiento y prueba. Guardelas en el archivo
# files/output/metrics.json. Cada fila del archivo es un diccionario con
# las metricas de un modelo. Este diccionario tiene un campo para indicar
# si es el conjunto de entrenamiento o prueba. Por ejemplo:
#
# {'type': 'metrics', 'dataset': 'train', 'r2': 0.8, 'mse': 0.7, 'mad': 0.9}
# {'type': 'metrics', 'dataset': 'test', 'r2': 0.7, 'mse': 0.6, 'mad': 0.8}
#


import json
from sklearn.metrics import r2_score, mean_squared_error, median_absolute_error


y_test_pred = model.predict(x_test)
y_train_pred = model.predict(x_train)


test_metrics = {
    "type": "metrics", 'dataset': 'test',
    'r2': r2_score(y_test, y_test_pred),
    'mse': mean_squared_error(y_test, y_test_pred),
    'mad': median_absolute_error(y_test, y_test_pred)
}




train_metrics = {

    "type": "metrics",   'dataset': 'train',
    'r2': r2_score(y_train, y_train_pred),
    'mse': mean_squared_error(y_train, y_train_pred),
    'mad': median_absolute_error(y_train, y_train_pred)
}
os.makedirs("../files/output", exist_ok=True)
with open("../files/output/metrics.json", 'w') as f:
    json.dump(train_metrics, f)
    f.write("\n")
    json.dump(test_metrics, f)